# Interacting with GPIO from MicroBlaze

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
import multiprocessing
import threading
import os
import socket

base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpio(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpio(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

//Add a C++ function to reset 0,1,4,5 LOW and 2,3,6,7 HIGH the GPIO pins on the chosen PMOD
void lh_gpio(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        if (j==0 || j==1 || j==4 || j==5) {
            write_gpio(j, 0);
        }
        else {
            write_gpio(j,1);
        }
    }
}

In [ ]:
LED_Status = [0, 0, 2, 3] #Default Status
#          = [off/on, solid/blink, cycle/blue/red/green, brightness]
LED_Change = 0 #this value does not matter. Will be the value of the element # in list user_input[]
user_input = []
press = False
Send_Status = False
compare_to = [0, 0, 1, 1, 0, 0, 1, 1] #this is if no buttons pressed!
iteration = -1 #To see where in the user_input array the specific input is remembered
user_input = [] #user_input[iteration] to see most recent addition.
sleep_time = 1
running = True
threads = []

def keypad(key_list = []):
    global compare_to, user_input, iteration
    lh_gpio()
    #find which key was pressed
    #print(key_list)
    if key_list[0] == 1:
        user_input.append('147*')
    elif key_list[1] == 1:
        user_input.append('2580')
    elif key_list[4] == 1:
        if key_list[2] == 0:
            user_input.append('3')
        elif key_list[3] == 0:
            user_input.append('6')
        else:
            user_input.append('9#')
    else: #key_list[5] == 1
        if key_list[2] == 0:
            user_input.append('A')
        elif key_list[3] == 0:
            user_input.append('B')
        else:
            user_input.append('CD')
    print('New Input Detected: ',user_input[iteration])
    
def button_press():
    global compare_to, iteration, key_list, press
    #see if a button was pressed
    value = []
    for i in range(8):
        value.append(read_gpio(i))
    if compare_to != value: #if a keypad input was detected
        iteration = iteration+1
        keypad(value) #find out which input was detected
        press = True
    #else: 

def send_status_change(val, el):
    global LED_Change, LED_Status
    print('Sent instructions to update array element ', el,' to a value of ',val)
    #send (val, el)
    
def Change_LED_Status():
    global compare_to, LED_Status, LED_Change, user_input, Send_Status
    LED_Status_New = []
    for i in range (4):
        LED_Status_New.append(LED_Status[i])
    
    if user_input[iteration] == '147*': #ON/OFF
        LED_Change = 0
        if LED_Status_New[0] == 0:
            LED_Status_New[0] = 1
        else:
            LED_Status_New[0] = 0
    elif user_input[iteration] == '2580': #Solid/Blink
        LED_Change = 1
        if LED_Status_New[1] == 0:
            LED_Status_New[1] = 1
        else:
            LED_Status_New[1] = 0
    elif user_input[iteration] == 'A': #RED
        LED_Change = 2
        LED_Status_New[2] = 2
    elif user_input[iteration] == 'B': #GREEN
        LED_Change = 2
        LED_Status_New[2] = 3
    elif user_input[iteration] == '3': #BLUE
        LED_Change = 2
        LED_Status_New[2] = 1
    elif user_input[iteration] == '6': #CYCLE
        LED_Change = 2
        LED_Status_New[2] = 0
    elif user_input[iteration] == 'CD': #BRIGHTER
        LED_Change = 3
        if LED_Status_New[3] < 3:
            LED_Status_New[3] = LED_Status[3]+1
    else: # (user_input[iteration]=='9#') #DIMMER
        LED_Change = 3
        if LED_Status_New[3] > 0:
            LED_Status_New[3] = LED_Status[3]-1
    
    #Now check to see if the button input changed LED state
    #(e.g. was brightness already max so does it need to send message to other pynq?)
    print('PREV LED Status: ', LED_Status)
    print('NEW LED STATUS:  ', LED_Status_New)
    if LED_Status != LED_Status_New:
        print('New status update to be sent')
        for i in range(4):
            LED_Status[i] = LED_Status_New[i]
        Send_Status = True
    else:
        print('User input does not result in any changes. Nothing will be sent.')

def LOOP(_l, num):
    global press, Send_Status, LED_Status, LED_Change, sleep_time, running
    while running:
        resource_available = _l.acquire(False)
        if resource_available == True:
            #print('Worker {} is searching for keypad input.'.format(num)) #USE TO FIND DEADLOCKS & adjust sleep time!
            button_press()
            if press == True:
                time.sleep(sleep_time) #add a delay so it doesn't register as multiple inputs!
                #change value of
                Change_LED_Status()
                if Send_Status == True:
                    send_status_change(LED_Status[LED_Change], LED_Change) #send the value of the list that was updated and the element num
                    Send_Status = False
                press = False
            else:
                time.sleep(sleep_time*.75) #MAKE SHORTER SO CAN READ SHORTER BUTTON INPUTS!
            lh_gpio()
            _l.release()
            time.sleep(sleep_time*.5)
        else:
            #INSERT FUNCTION TO SEE IF NEW MESSAGE CAME IN
            #print('Worker {} is searching for incoming message.'.format(num)) #USE TO FIND DEADLOCKS & adjust sleep time!
            time.sleep(sleep_time*1.5)

def Start_Keypad():    
    global threads
    lh_gpio() #reset gpio to same as "compare_to"
    fork = threading.Lock()
    for i in range(2):
        #fork is the shared lock
        #i is the user (led number too)
        t = threading.Thread(target=LOOP, args=(fork, i))
        threads.append(t) #creates the new thread number
        t.start()
        
def Stop_Keypad():
    global running, threads
    running = False
    for t in threads:
        t.join()

In [ ]:
def run_pynq_server_C():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('192.168.2.99', 54321))
    sock.listen()
    conn, addr = sock.accept()
    with conn:
        print("before pynq C")
        while True:
            print("inside pynq C")
            time.sleep(0.0001)
            data = conn.recv(1024)
            if (data.decode() == 'disconnect'):
                print("disconnect pynq C")
                #Stop_RGB()
                break
    print("after pynq C")

def run_pynq_client_C():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    while True:
        time.sleep(0.0001)
        if base.buttons[0].read() != 0:
            sock.connect(('192.168.2.1', 12345))
            break
    time.sleep(0.0001)
    Start_Keypad()
    sock.sendall(b'RGB') # TODO: replace with function to regularly send corresponding input to server_C
    while True:
        time.sleep(0.0001)
        if base.buttons[3].read() != 0:
            sock.sendall(b'disconnect')
            Stop_Keypad()
            break
    sock.close()

p_server = multiprocessing.Process(target=run_pynq_server_C)
p_server.start()
os.system("taskset -p -c {} {}".format(0, p_server.pid))

p_client = multiprocessing.Process(target=run_pynq_client_C)
p_client.start()
os.system("taskset -p -c {} {}".format(1, p_client.pid))

p_server.join();
p_client.join();